https://pytorch.org/tutorials/index.html

https://yutaroogawa.github.io/pytorch_tutorials_jp/


ここの内容を噛み砕いたものが多いのでわからない場合はここを見てください

machine learning
====================

ここに置いてあるnotebookを全部見た後にもっかいみたら何が書いてあるかかなりわかるようになると思う．

雰囲気をざっくり書いておく．

In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from typing import Any, Callable, Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader


import time
from torch.optim.optimizer import Optimizer
from typing import List
from torch import Tensor
import numpy

Cutout
===============

データ拡張の一つ
汎化性能を大きく向上させてくれる

In [ ]:
class Cutout:
    def __init__(self, size=16, p=0.5):
        self.size = size
        self.half_size = size // 2
        self.p = p

    def __call__(self, image):
        if torch.rand([1]).item() > self.p:
            return image

        left = torch.randint(-self.half_size, image.size(1) - self.half_size, [1]).item()
        top = torch.randint(-self.half_size, image.size(2) - self.half_size, [1]).item()
        right = min(image.size(1), left + self.size)
        bottom = min(image.size(2), top + self.size)

        image[:, max(0, left): right, max(0, top): bottom] = 0
        return image

データセット
=============

今回の学習にはCifar10を使ってる


In [ ]:
class Cifar10:
    def __init__(self, batch_size, threads):
        mean, std = self._get_statistics()

        train_transform = transforms.Compose([
            torchvision.transforms.RandomCrop(size=(32, 32), padding=4),
            torchvision.transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std),
            Cutout()
        ])

        test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
        test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

        self.train = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=threads)
        self.test = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=threads)

        self.classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    def _get_statistics(self):
        train_set = torchvision.datasets.CIFAR10(root='./cifar', train=True, download=True, transform=transforms.ToTensor())

        data = torch.cat([d[0] for d in DataLoader(train_set)])
        return data.mean(dim=[0, 2, 3]), data.std(dim=[0, 2, 3])

ハイパーパラメータ
================

自分で調整する量

今回は学習の負担を軽くするためにepoch数を小さくしてる

普通ならepochs = 200くらいになる．

In [ ]:
###############
# 入力パラメータ
###############
#バッチサイズ
batch_size = 128
#エポック数
epochs = 10
#GPUID
ngpu = 1
#学習率
lr = 0.1
#スレッド数
threads = 2

#device = torch.device("cuda:6" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
use_cuda = torch.cuda.is_available()
if use_cuda:
    torch.cuda.empty_cache()
    torch.cuda.set_device(0)

In [ ]:
dataset = Cifar10(batch_size,threads)
train_data = dataset.train
test_data = dataset.test

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


モデル
============

ネットワークを定義している

今回は学習の計算の負担を軽くするために小さなモデルを使ってる

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model=Net().cuda()

最適化手法
============

最適化手法の定義をしている

今回使っているのはSGDたくさんのオプションがあるので気になる人はググって使うといい
https://pytorch.org/docs/stable/generated/torch.optim.SGD.html

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

損失関数
-------------------------
損失関数の定義をしている

criterionとすることが多い．

これもたくさんの関数があるので気になる人はググると良い

今回はクロスエントロピー誤差を使う．

In [ ]:
criterion = nn.CrossEntropyLoss()

学習率減衰
==================

学習を進めていく過程で常に同じ学習率だと停滞が起こる．

そこで一定のエポック数で学習率を小さくしてあげる必要がありここで定義した関数で学習率を操作する．

今回は総エポック数が小さいので動作しない(epochが20ごとに0.5倍)が後学のために載せる．

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

訓練
==================

訓練を行う関数

基本的な流れとしては

1「入力」を「モデル」に入れて「ラベル」との「損失」を出す

2「損失」の勾配計算を行い，パラメータ更新を行う

3「学習率減衰」と「精度」や「損失値」の計算を行う

といった流れ

In [ ]:
def train(epoch):
    model.train()
    train_loss = 0.0
    correct = 0.0

    for inputs, target in dataset.train:
        optimizer.zero_grad()
        input_size = inputs.size()[0]
        inputs, target = inputs.cuda(), target.cuda()
        output = model(inputs)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if epoch >= args.start_averaged:
            averaged_model.update_parameters(model)
        with torch.no_grad():
            train_loss += loss.data * input_size/total_train
            _, pred = torch.max(output, 1)
            correct += pred.eq(target).sum()/total_train


    return train_loss, 100*correct

予測
==================

予測を行う関数

基本的な流れとしては

1「入力」を「モデル」に入れて「ラベル」との「損失」を出す

2「精度」や「損失値」の計算を行う

といった流れ

訓練の時と違ってパラメータ更新を行う必要がないので勾配計算は行わない

In [ ]:
def test(epoch):
    loss_epoch = 0.0
    acc_epoch = 0.0

    with torch.no_grad():
        for inputs,labels in test_data:
          #1「入力」を「モデル」に入れて「ラベル」との「損失」を出す
          labels = labels.cuda()
          inputs = inputs.cuda()
          outputs = model(inputs)
          loss = criterion(outputs,labels)
          _,preds = torch.max(outputs,1)

          #2「精度」や「損失値」の計算を行う
          loss_epoch += loss.item() * inputs.size(0)
          acc_epoch += torch.sum(preds == labels.detach())

        loss_epoch = loss_epoch / len(test_data.dataset)
        acc_epoch = acc_epoch.double() / len(test_data.dataset)
        return loss_epoch,acc_epoch

def test(epoch):
    model.eval()
    averaged_model.eval()

    test_loss = 0.0
    test_correct = 0.0

    with torch.no_grad():
        for inputs, target in dataset.test:
            input_size = inputs.size()[0]
            inputs, target = inputs.cuda(), target.cuda()
            #normal_prediction
            predict = model(inputs)
            loss = loss_func(predict, target)
            test_loss += loss.data * input_size / total_test
            _, pred = torch.max(predict.data, 1)
            test_correct += pred.eq(target).sum() / total_test

    return test_loss, 100*test_correct

メイン
==================

定義してきた関数を使って学習と予測をエポック数分行う．


In [ ]:
for epoch in range(epochs):

    torch.cuda.synchronize()
    start = time.time()
    model.train()

    train_loss,train_acc = train(epoch)

    torch.cuda.synchronize()
    train_time = time.time() - start

    print(f"epoch:{epoch}",
            f"train_loss:{train_loss:.4f}",
            f"train_acc:{train_acc:.4f}",
            f"time:{train_time:.4f}")

    model.eval()
    test_loss,test_acc = test(epoch)

    torch.cuda.synchronize()
    test_time = time.time() - start

    print(f"epoch:{epoch}",
            f"test_loss:{test_loss:.4f}",
            f"test_acc:{test_acc:.4f}",
            f"time:{test_time:.4f}")


print('Finished Training')

epoch:0 train_loss:2.2492 train_acc:0.1667 time:19.5128
epoch:0 test_loss:2.2320 test_acc:0.1841 time:21.8332
epoch:1 train_loss:2.2417 train_acc:0.1772 time:17.5058
epoch:1 test_loss:2.2320 test_acc:0.1841 time:19.7373
epoch:2 train_loss:2.2416 train_acc:0.1757 time:17.1860
epoch:2 test_loss:2.2320 test_acc:0.1841 time:19.5927
epoch:3 train_loss:2.2420 train_acc:0.1766 time:16.4482
epoch:3 test_loss:2.2320 test_acc:0.1841 time:18.3899
epoch:4 train_loss:2.2419 train_acc:0.1762 time:16.7059
epoch:4 test_loss:2.2320 test_acc:0.1841 time:18.6992
epoch:5 train_loss:2.2412 train_acc:0.1771 time:16.4898
epoch:5 test_loss:2.2320 test_acc:0.1841 time:18.4455
epoch:6 train_loss:2.2417 train_acc:0.1772 time:17.2668
epoch:6 test_loss:2.2320 test_acc:0.1841 time:20.1913
epoch:7 train_loss:2.2415 train_acc:0.1771 time:19.8106
epoch:7 test_loss:2.2320 test_acc:0.1841 time:22.2965
epoch:8 train_loss:2.2421 train_acc:0.1758 time:17.9071
epoch:8 test_loss:2.2320 test_acc:0.1841 time:20.4635
epoch:9 tr